# Q.6 CNN with SGD, MC Dropout, and Epistemic Uncertainty
Train a controlled convolutional neural network (CNN) on a subset of the SVHN dataset using SGD optimizer. Then, apply Monte Carlo (MC) Dropout at inference to estimate both test accuracy and epistemic uncertainty. Set random seeds to 42. Use the following configuration:
- Load SVHN and normalize pixel values to [0,1]
- Use only the first 2000 training samples and first 500 test samples
- Input shape: 32 × 32 × 3
- CNN architecture:
- Conv2D: 32 filters, 3×3 kernel, ReLU activation
- MaxPooling2D: 2×2
- Conv2D: 64 filters, 3×3 kernel, ReLU activation
- MaxPooling2D: 2×2
- Flatten
- Dense: 128 neurons, ReLU activation
- Dropout: 0.25 (keep during inference for MC Dropout)
- Output layer: 10 neurons with softmax
- Optimizer: SGD with momentum = 0.9, learning rate = 0.01
- Loss: sparse categorical crossentropy
- epochs = 15, batch size = 32
- For MC Dropout:
- Enable dropout during inference
- Average predictions over 20 stochastic forward passes
- Compute the epistemic uncertainty as the predictive variance across passes

Q6.1 Report the plain test accuracy of the CNN trained with SGD (no MC Dropout).

Q6.2 Report the MC Dropout-enhanced accuracy (averaging 20 stochastic predictions).

Q6.3 Compute the average epistemic uncertainty (mean predictive variance) across all test samples. Report it as a deterministic number rounded to 3 decimal places


# Imports, seed and data loading


In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import SVHN
import numpy as np, random

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.ToTensor()

train_full = SVHN('./data', split='train', download=True, transform=transform)
test_full = SVHN('./data', split='test', download=True, transform=transform)

train_subset = Subset(train_full, range(2000))
test_subset = Subset(test_full, range(500))

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

# CNN definition
(this is gonna be the pre-MC model)

In [37]:
class CNN_SVHN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 128), nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.classifier(self.features(x))

# Model, optimizer, loss
model = CNN_SVHN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

Epochs = 15
for epoch in range(Epochs):
    model.train()
    running_loss = 0
    correct, total = 0, 0  # reset each epoch
    
    for x, y in train_dl:
        x, y = x.to(device), y.to(device).long()
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        preds = out.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{Epochs}, Loss: {running_loss/len(train_dl):.4f}, Accuracy: {acc:.2f}%")


Epoch 1/15, Loss: 2.2439, Accuracy: 20.20%
Epoch 2/15, Loss: 2.2325, Accuracy: 20.40%
Epoch 3/15, Loss: 2.2279, Accuracy: 20.40%
Epoch 4/15, Loss: 2.2260, Accuracy: 20.40%
Epoch 5/15, Loss: 2.2227, Accuracy: 20.40%
Epoch 6/15, Loss: 2.2199, Accuracy: 20.40%
Epoch 7/15, Loss: 2.2183, Accuracy: 20.40%
Epoch 8/15, Loss: 2.2142, Accuracy: 20.40%
Epoch 9/15, Loss: 2.2064, Accuracy: 20.40%
Epoch 10/15, Loss: 2.1948, Accuracy: 20.40%
Epoch 11/15, Loss: 2.1812, Accuracy: 20.30%
Epoch 12/15, Loss: 2.1639, Accuracy: 20.40%
Epoch 13/15, Loss: 2.1220, Accuracy: 22.80%
Epoch 14/15, Loss: 2.0645, Accuracy: 25.20%
Epoch 15/15, Loss: 1.9714, Accuracy: 32.30%


### Q6.1 
The accuracy is low because the model doesn’t have enough data or depth to really learn the complex patterns in the SVHN images. It’s only trained on 2000 samples, and the network is pretty simple. Also, using plain SGD makes learning slower, so it doesn’t reach good performance within just 15 epochs. ( i think WIP)

### Q6.2 Report the MC Dropout-enhanced accuracy (averaging 20 stochastic predictions).



In [38]:
def mc_dropout_predictions(model, x, passes=20):
    """MC Dropout predictions: keep dropout active during inference."""
    model.train()  # ENABLE dropout
    preds = []
    for _ in range(passes):
        preds.append(torch.softmax(model(x), dim=1).unsqueeze(0))
    return torch.cat(preds, dim=0)  # [passes, batch, classes]

correct_mc, total_mc = 0, 0
with torch.no_grad():
    for x, y in test_dl:
        x, y = x.to(device), y.to(device)
        preds_mc = mc_dropout_predictions(model, x, passes=20)
        mean_preds = preds_mc.mean(0)
        final_preds = mean_preds.argmax(1)
        correct_mc += (final_preds == y).sum().item()
        total_mc += y.size(0)

mc_acc = 100 * correct_mc / total_mc
print(f"MC Dropout-enhanced Accuracy: {mc_acc:.2f}%")


MC Dropout-enhanced Accuracy: 28.80%


## Q6.3 Compute the average epistemic uncertainty (mean predictive variance) across all test samples. Report it as a deterministic number rounded to 3 decimal places

### Whats epistemic uncertainty?
Epistemic uncertainty is the uncertainty in a model’s predictions due to lack of knowledge or limited training data. It reflects how unsure the model is about its predictions and can be reduced by providing more data or improving the model. 

In practice, MC Dropout estimates it by measuring variability in predictions across multiple stochastic forward passes. High epistemic uncertainty means the model is less confident, while low uncertainty means it is more confident in its outputs.

In [43]:

import numpy as np

epistemic_vars = []

with torch.no_grad():
    for x, _ in test_dl:
        x = x.to(device)
        preds_mc = mc_dropout_predictions(model, x, passes=20)  # [20, batch, 10]
        # variance per sample over stochastic passes
        var_preds = preds_mc.var(0).mean(1)  # mean over classes
        epistemic_vars.extend(var_preds.cpu().numpy())

avg_epistemic_uncertainty = np.mean(epistemic_vars)
print(f"Average Epistemic Uncertainty: {round(avg_epistemic_uncertainty, 3)}")
print("Just showing that its very low:")
print(f"Average Epistemic Uncertainty: {round(avg_epistemic_uncertainty, 10)}")

Average Epistemic Uncertainty: 0.0
Just showing that its very low:
Average Epistemic Uncertainty: 0.0004344634071458131


The model has a very low epistemic uncertainty, so much so that rounding it up to 3 decimals will only display 0. This means the model is extremely confident in its predictions. but given its low test accuracy, this confidence is likely wrong, the model is consistently making similar predictions, even when they are wrong. This can be because it may be underfitting or not learning new representations from the limited training data.